In [2]:
using DifferentialEquations, Plots
using DifferentialEquations.EnsembleAnalysis  # Needed? just to make sure we have acces...

In [4]:
using StatsBase, DataFrames

In [3]:
# set up DE
function μ_SDE(du,u,p,t)
  du .= p.μ
end

function σ_SDE(du,u,p,t)
  du .= p.σ
end

σ_SDE (generic function with 1 method)

In [83]:
T = 10.;

function save_func(u, t, integrator) 
    moments = [minimum(u), mean(u), maximum(u)]
    push!(integrator.p.moments, moments) 
end # save everything for now 

p = (μ = 0.01, σ = 0.1, N = 3, moments = Array{Array{Float64, 1}, 1}[]);
prob = SDEProblem(μ_SDE, σ_SDE, x_iv ,(0.0, T), p);
saveat = 0:0.1:T

cb = FunctionCallingCallback(save_func;
                 funcat=saveat,
                 func_everystep=true,
                 func_start = true,
                 tdir=1);

In [84]:
function output_func(sol, i)
    return (u = sol.prob.p.moments, t = sol.t), false
end

output_func (generic function with 1 method)

In [85]:
function prob_func(prob,i,repeat)
    p = (μ = 0.01, σ = 0.1, N = 3, moments = Array{Array{Float64, 1}, 1}[]);
  SDEProblem(μ_SDE, σ_SDE, x_iv ,(0.0, T), p);
end

prob_func (generic function with 1 method)

In [86]:
ensemble_prob = EnsembleProblem(prob, prob_func = prob_func, output_func = output_func)
sim = solve(ensemble_prob,Tsit5(), EnsembleSerial(), trajectories = 2, callback = cb)

MethodError: MethodError: Cannot `convert` an object of type Float64 to an object of type Array{Float64,1}
Closest candidates are:
  convert(::Type{Array{Float64,1}}, !Matched::Sundials.NVector) at /Users/arnavsood/.julia/packages/Sundials/OdI66/src/nvector_wrapper.jl:56
  convert(::Type{Array{Float64,1}}, !Matched::Ptr{Sundials._generic_N_Vector}) at /Users/arnavsood/.julia/packages/Sundials/OdI66/src/nvector_wrapper.jl:77
  convert(::Type{Array{T,N}}, !Matched::FillArrays.Zeros{V,N,Axes} where Axes) where {T, V, N} at /Users/arnavsood/.julia/packages/FillArrays/MoGDs/src/FillArrays.jl:301
  ...

In [87]:
sim.u[1]

(u = Any[[0.0474938, 0.306015, 0.590772], [0.0474938, 0.306015, 0.590772], [0.0484601, 0.305747, 0.586085], [0.0538505, 0.307969, 0.585297], [0.0526503, 0.306802, 0.584087], [0.0588576, 0.309676, 0.580334], [0.0561227, 0.309885, 0.589188], [0.0511345, 0.306592, 0.589823], [0.0551127, 0.314391, 0.59902], [0.056093, 0.318733, 0.609295]  …  [-0.658448, 0.181607, 0.657987], [-0.656041, 0.182428, 0.657615], [-0.647262, 0.185421, 0.656258], [-0.638483, 0.188413, 0.654902], [-0.637754, 0.188662, 0.65479], [-0.671431, 0.180952, 0.653748], [-0.708159, 0.172543, 0.652612], [-0.724256, 0.168858, 0.652114], [-0.719923, 0.157205, 0.616711], [-0.719923, 0.157205, 0.616711]], t = [0.0, 0.00145079, 0.00282616, 0.00437344, 0.00611413, 0.00807241, 0.0102755, 0.0127539, 0.0155422, 0.018679  …  8.08723, 8.31124, 8.54033, 8.77178, 9.00487, 9.23802, 9.47258, 9.70831, 9.94383, 10.0])

In [88]:
summ = EnsembleSummary(sim)

MethodError: MethodError: no method matching zero(::Type{Any})
Closest candidates are:
  zero(::Type{Union{Missing, T}}) where T at missing.jl:87
  zero(!Matched::Type{LibGit2.GitHash}) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.1/LibGit2/src/oid.jl:220
  zero(!Matched::Type{Pkg.Resolve.VersionWeights.VersionWeight}) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.1/Pkg/src/resolve/VersionWeights.jl:19
  ...